In [ ]:
import time

# Record start time
start_time = time.time()


import nltk
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer#reducing words to their base form.
from nltk.tokenize import TweetTokenizer
import re
import string
import pdb

nltk.download('stopwords')
nltk.download('twitter_samples')


end_time = time.time()

# Calculate total time
total_time = end_time - start_time

# Print the total time
print(f"Total time taken: {total_time} seconds")

In [ ]:
import time

# Record start time
start_time = time.time()






#FUNTIONS:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*|^RT[\s]+|https?://[^\s\n\r]+|#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    tweets_clean = []
    for word in tweet_tokens:
        if(word not in stopwords_english and word not in string.punctuation):
            stem_word = stemmer.stem(word)
            tweets_clean.append(stem_word)
    return tweets_clean
    
def count_tweets(tweets, ys):
    ys_list = np.squeeze(ys).tolist()
    freqs = {}
    for y,tweet in zip(ys_list, tweets):
        for word in process_tweet(tweet):
            pair = (word,y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

def train_naive_bayes(freqs, train_x,train_y):
    logprior = 0
    loglikelihood = {}
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    N_pos=0
    N_neg=0
    for pair in freqs.keys():
        if pair[1]>0:
            N_pos += 1
        else:
            N_neg += 1
    
    D = train_y.shape[0]
    D_pos = train_y[train_y==1].shape[0]
    D_neg = train_y[train_y==0].shape[0]
    
    logprior = np.log(D_pos) - np.log(D_neg)
    
    for word in vocab:
        freqs_pos = freqs.get((word,1),0)
        freqs_neg = freqs.get((word,0),0)
        
        p_w_pos = (freqs_pos + 1)/(N_pos + V)
        p_w_neg = (freqs_neg + 1)/(N_neg + V)

        loglikelihood[word] = np.log(p_w_pos) - np.log(p_w_neg)
    
    return logprior, loglikelihood

def predict_tweet(tweet):
    logprior, loglikelihood = train_naive_bayes(freqs, train_x,train_y)
    word_l = process_tweet(tweet)
    p = 0
    p += logprior
    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]
    return p

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]
train_x = train_pos + train_neg 
test_x = test_pos + test_neg
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

freqs = count_tweets(train_x,train_y)


#prediction
my_tweet = input("Enter a tweet: ")

p = predict_tweet(my_tweet)
print(p)
if p>0.5999:
    print("Positive Sentiment")
elif p>=-0.5999 and p<=0.5999:
    print("Neutral Sentiment")
else:
    print("Negative Sentiment")

    
    
end_time = time.time()

# Calculate total time
total_time = end_time - start_time

# Print the total time
print(f"Total time taken: {total_time} seconds")